<a href="https://colab.research.google.com/github/meiruv/Resumes_Dataset/blob/main/Field_Classification_With_Community_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Import

In [1]:
import json

KAGGLE_PROJECT_NAME = 'gauravduttakiit/resume-dataset'

TO_KAGGLE_JSON = {'username': 'meiravu', 'key': 'a3bfa698c6ebd7f58afcb901af24870a'}

with open('kaggle.json', 'w') as json_file:
  json.dump(TO_KAGGLE_JSON, json_file)

! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! cp kaggle.json ~/.kaggle/
! kaggle datasets download -d $KAGGLE_PROJECT_NAME



  0% 0.00/383k [00:00<?, ?B/s]
100% 383k/383k [00:00<00:00, 51.5MB/s]


In [2]:
import os
import zipfile

local_zip = 'resume-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('resume-dataset')
zip_ref.close()

# Loading the Data

In [3]:
import pandas as pd

resume_df = pd.read_csv('resume-dataset/UpdatedResumeDataSet.csv')
resume_df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [4]:
resume_df['Category'].value_counts()

Java Developer               84
Testing                      70
DevOps Engineer              55
Python Developer             48
Web Designing                45
HR                           44
Hadoop                       42
ETL Developer                40
Blockchain                   40
Sales                        40
Data Science                 40
Mechanical Engineer          40
Operations Manager           40
Arts                         36
Database                     33
Electrical Engineering       30
Health and fitness           30
PMO                          30
DotNet Developer             28
Business Analyst             28
Automation Testing           26
Network Security Engineer    25
SAP Developer                24
Civil Engineer               24
Advocate                     20
Name: Category, dtype: int64

# PreProcessing

In [5]:
# Cleaning the texts
from gensim.parsing.preprocessing import preprocess_string, remove_stopwords, strip_multiple_whitespaces, strip_non_alphanum, strip_short, strip_numeric, stem_text

resume_df['clean_resume'] = resume_df.apply(lambda row: strip_multiple_whitespaces(strip_non_alphanum(remove_stopwords(strip_numeric(row['Resume'].lower().encode('ascii',errors='ignore'))))), axis = 1)


In [6]:
resume_df['clean_resume']

0      skills programming languages python pandas num...
1      education details b e uit rgpv data scientist ...
2      areas deep learning control design programming...
3      skills r python sap hana tableau sap hana sql ...
4      education details mca ymcaust faridabad haryan...
                             ...                        
957    skills proficient ms office word basic excel p...
958    willingness accept challenges positive thinkin...
959    personal skills quick learner eagerness learn ...
960    skills software knowledge ms power point ms of...
961    skill set os windows xp database mysql sql ser...
Name: clean_resume, Length: 962, dtype: object

In [7]:
from gensim.parsing.preprocessing import preprocess_string

# I removed any charachter that isn't in english via encoding to ascii, stripped numbers and punctuation because I only care for
# words here and dropped stop words as they don't contribute to my purpose and only generate noise.
# I considered removing non english words and stemming but this led to ruining words like names of packages (
# for example - stemming turned pandas into panda) so I decided to not do this at the moment and perhaps later figure
# out how to unite words like design and designed without harming names and domain phrases

CUSTOM_FILTERS = [lambda x: x.lower().encode('ascii',errors='ignore'), strip_multiple_whitespaces, strip_non_alphanum, remove_stopwords,strip_numeric]

resume_df['clean_resume_tokens'] = resume_df.apply(lambda row: preprocess_string(row['Resume'], CUSTOM_FILTERS), axis = 1)



In [8]:
resume_df.loc[0,'clean_resume_tokens']

['skills',
 'programming',
 'languages',
 'python',
 'pandas',
 'numpy',
 'scipy',
 'scikit',
 'learn',
 'matplotlib',
 'sql',
 'java',
 'javascript',
 'jquery',
 'machine',
 'learning',
 'regression',
 'svm',
 'nave',
 'bayes',
 'knn',
 'random',
 'forest',
 'decision',
 'trees',
 'boosting',
 'techniques',
 'cluster',
 'analysis',
 'word',
 'embedding',
 'sentiment',
 'analysis',
 'natural',
 'language',
 'processing',
 'dimensionality',
 'reduction',
 'topic',
 'modelling',
 'lda',
 'nmf',
 'pca',
 'neural',
 'nets',
 'database',
 'visualizations',
 'mysql',
 'sqlserver',
 'cassandra',
 'hbase',
 'elasticsearch',
 'd',
 'js',
 'dc',
 'js',
 'plotly',
 'kibana',
 'matplotlib',
 'ggplot',
 'tableau',
 'regular',
 'expression',
 'html',
 'css',
 'angular',
 'logstash',
 'kafka',
 'python',
 'flask',
 'git',
 'docker',
 'vision',
 'open',
 'cv',
 'understanding',
 'deep',
 'learning',
 'education',
 'details',
 'data',
 'science',
 'assurance',
 'associate',
 'data',
 'science',
 'assur

# Counting Appearance of a Word in Each Resume Via Bag Of Words

In [9]:
from gensim import corpora

dictionary = corpora.Dictionary(resume_df['clean_resume_tokens'])
print(dictionary)

Dictionary(7082 unique tokens: ['accelerating', 'accounting', 'action', 'addresses', 'advanced']...)


In [10]:
from gensim.parsing.preprocessing import stem_text

stem_text('designed designing')

'design design'

In [11]:
dictionary.token2id

{'accelerating': 0,
 'accounting': 1,
 'action': 2,
 'addresses': 3,
 'advanced': 4,
 'analysis': 5,
 'analytic': 6,
 'analytics': 7,
 'analyzing': 8,
 'angular': 9,
 'anomalies': 10,
 'answer': 11,
 'asks': 12,
 'assisted': 13,
 'assisting': 14,
 'assists': 15,
 'associate': 16,
 'assurance': 17,
 'attacks': 18,
 'automated': 19,
 'automating': 20,
 'based': 21,
 'bayes': 22,
 'best': 23,
 'blob': 24,
 'boosting': 25,
 'bootstrap': 26,
 'bot': 27,
 'build': 28,
 'case': 29,
 'cases': 30,
 'cassandra': 31,
 'categories': 32,
 'category': 33,
 'chain': 34,
 'chat': 35,
 'chatbot': 36,
 'classification': 37,
 'classifier': 38,
 'clients': 39,
 'cloud': 40,
 'cluster': 41,
 'coding': 42,
 'comments': 43,
 'company': 44,
 'content': 45,
 'core': 46,
 'cosine': 47,
 'costs': 48,
 'counter': 49,
 'created': 50,
 'css': 51,
 'customer': 52,
 'customized': 53,
 'cv': 54,
 'cyber': 55,
 'd': 56,
 'dashboard': 57,
 'dashboards': 58,
 'data': 59,
 'database': 60,
 'dc': 61,
 'decision': 62,
 'dec

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(lowercase = False, min_df = 15, max_df = 900)
X = vectorizer.fit_transform(resume_df['clean_resume'])


In [13]:
vectorizer.get_feature_names()

['ab',
 'abilities',
 'ability',
 'able',
 'academic',
 'academy',
 'acb',
 'accenture',
 'accept',
 'acceptance',
 'accepted',
 'access',
 'accordance',
 'according',
 'account',
 'accounting',
 'accounts',
 'accuracy',
 'accurate',
 'accurately',
 'ace',
 'achieve',
 'achieved',
 'achievements',
 'achieving',
 'acquired',
 'acquiring',
 'act',
 'action',
 'actions',
 'active',
 'actively',
 'activities',
 'activity',
 'actual',
 'actuator',
 'actuators',
 'ad',
 'add',
 'added',
 'adding',
 'additional',
 'additionally',
 'address',
 'addressed',
 'adherence',
 'admin',
 'administration',
 'administrative',
 'administrator',
 'advance',
 'advanced',
 'advantage',
 'advertising',
 'advocate',
 'aes',
 'agent',
 'agents',
 'agile',
 'agreement',
 'aim',
 'air',
 'airtel',
 'aissms',
 'aix',
 'ajax',
 'alarm',
 'alert',
 'alerts',
 'algorithm',
 'algorithms',
 'allow',
 'also',
 'alternate',
 'american',
 'amravati',
 'analyse',
 'analyses',
 'analysis',
 'analyst',
 'analysts',
 'analy

In [14]:
bow_df = pd.DataFrame(X.toarray())
bow_df.columns = vectorizer.get_feature_names()
bow_df.head()

,ab,abilities,ability,able,academic,academy,acb,accenture,accept,acceptance,accepted,access,accordance,according,account,accounting,accounts,accuracy,accurate,accurately,ace,achieve,achieved,achievements,achieving,acquired,acquiring,act,action,actions,active,actively,activities,activity,actual,actuator,actuators,ad,add,added,...,webdriver,website,websites,weekly,weeks,well,wide,willing,willingness,win,window,windows,wipro,wiring,women,word,work,worked,workflow,working,works,workshop,workshops,world,worldwide,write,writing,written,wrote,www,xen,xillinx,xml,xp,year,years,york,yrs,zensar,zone
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0
3,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0


In [15]:
bow_df.shape

(962, 2307)

# Filtering the words so we only keep meaningful ones

In [ ]:
0.01 * 962

# Turning To Graph